# Djikstra's Algorithm
If you want a really nice refresher on Djikstra's Algortith, check out this [Computerphile video](https://youtu.be/GazC3A4OQTE). 

Djikstra's finds the shortest path between two nodes. The essence of the algorithm is as follows:
* Start with node "a"
* Add children to priority queue, prioritizing by the length of the total path from "a" to this point.
* Keep going unti the priority queue is empty (no path exists) or b is at the top of your queue.

To do this, you need a special kind of priority queue that allows you that allows you to update an element's priority if a shorter path is found. I created a class called `MutableHeap` to handle this complexity for me, as well as a `Node` class for the graph representation.

In [1]:
import heapq
import math
import itertools
from collections import namedtuple

class Node:
    def __init__(self, position):
        self.position = position
        self.neighbors = {}
    def __str__(self):
        return self.position
    def __repr__(self):
        return str(self.position)
    def add_neighbors(self, neighbors):
        for neighbor, weight in neighbors.items():
            self.neighbors[neighbor] = weight

# Used to represent a path from the start to the given node
Path = namedtuple("Path", ("length", "node", "prev_node"))

class MutableHeap:
    def __init__(self):
        self.__paths = []
        self.__nodes = {}
        self.__count = itertools.count()
    def __len__(self):
        return len(self.__paths)
    def push(self, path, priority):
        if type(path) != Path:
            raise TypeError("Must be a Path namedtuple!")
        # Check to update node if it already exists
        if path.node in self.__nodes:
            cur_entry = self.__nodes[path.node]
            if path.length > cur_entry[2].length:
                # Current path from start is better.
                return
            cur_entry[2] = None
        entry = [priority, next(self.__count), path]
        self.__nodes[path.node] = entry
        heapq.heappush(self.__paths, entry)
    def pop(self):
        pop = None
        while len(self.__paths) and pop == None:
            pop = heapq.heappop(self.__paths)[2]
        if pop:
            self.__nodes.pop(pop.node)
        return pop

Next, I wrote the meat of the algorithm. With the ugliness of the priority queue out of the way, it's actually pretty simple.

In [2]:
def djikstra(a, b):
    visited = {}
    heap = MutableHeap()
    heap.push(Path(0, a, None), 0)
    while len(heap):
        shortest_path = heap.pop()
        print("Processing", shortest_path.node.position)
        for n, length in shortest_path.node.neighbors.items():
            if n not in visited:
                print("--adding:", n.position, "to heap")
                new_path = Path(shortest_path.length + length, n, shortest_path.node)
                heap.push(new_path, new_path.length)
        visited[shortest_path.node] = shortest_path.prev_node
        if shortest_path.node == b:
            return get_path(b, visited) #Path found!
    return None #No path found.

def get_path(node, path_to):
    path = []
    cur_node = node
    while cur_node:
        path.append(cur_node)
        cur_node = path_to[cur_node]
    return path[::-1]

# A*
A problem with Djikstra's algorithm is that it *always* goes with the shortest path first, even if that path is taking you farther away from your goal. A\* fixes that by adding a heuristic that nodes closer to your target are more likely to lead you to your shortest path. Therefore, instead of prioritizing by just the length of the path, you prioritize by the length + the distance you are from your goal. 

This is particularly useful when thought of geographically. If each node has a coordinate of where it is in space, you can compute the distance between those two nodes. Then the shorter paths with smaller distances get picked first, and it allows your algorithm to more or less beeline for your endpoint.

It's worth noting that these algorithms are practically the same. The only difference between my `a_star` algorithm and my `djikstra` algorithm is when pushing onto the heap (line 12), you set the priority to `length + distance_to_b` instead of just the length.

In [3]:

def a_star(a, b):
    visited = {}
    heap = MutableHeap()
    heap.push(Path(0, a, None), get_distance(a, b))
    while len(heap):
        best_path = heap.pop()
        print("Processing", best_path.node.position)
        for n, length in best_path.node.neighbors.items():
            if n not in visited:
                print("--adding:", n.position, "to heap")
                new_path = Path(best_path.length + length, n, best_path.node)
                distance_to_b = get_distance(n, b)
                heap.push(new_path, distance_to_b)
            visited[best_path.node] = best_path.prev_node
            if best_path.node == b:
                return get_path(b, visited) #Path found!
    return None #No path found.

def get_distance(a, b):
    x1, y1 = a.position
    x2, y2 = b.position
    xdiff = (x2 - x1) ** 2   
    ydiff = (y2 - y1) ** 2
    return math.sqrt(xdiff + ydiff)

# Testing it Out


In [6]:
nodes = [Node((x, y)) for x in range(4) for y in range(4)]

nodes[0].add_neighbors({nodes[1]:2, nodes[5]:2})
nodes[1].add_neighbors({nodes[0]:2, nodes[5]:1})
nodes[2].add_neighbors({nodes[3]:3})
nodes[3].add_neighbors({nodes[2]:3, nodes[7]:3})

nodes[4].add_neighbors({nodes[8]:4})
nodes[5].add_neighbors({nodes[0]:2, nodes[1]:1, nodes[10]:1})
nodes[6].add_neighbors({nodes[7]:1, nodes[10]:2})
nodes[7].add_neighbors({nodes[3]:3, nodes[6]:1, nodes[11]:2})

nodes[8].add_neighbors({nodes[4]:4, nodes[9]:3})
nodes[9].add_neighbors({nodes[8]:3, nodes[10]:2, nodes[13]:1})
nodes[10].add_neighbors({nodes[5]:1, nodes[6]:2, nodes[9]:2, nodes[15]:1})
nodes[11].add_neighbors({nodes[7]:2})

nodes[12].add_neighbors({nodes[13]:2})
nodes[13].add_neighbors({nodes[12]:2, nodes[14]:3, nodes[9]:1})
nodes[14].add_neighbors({nodes[15]:1, nodes[13]:3})
nodes[15].add_neighbors({nodes[14]:1, nodes[10]:1})

print("****Djikstra****")
print(djikstra(nodes[0], nodes[15]))
print("\n****A Star****")
print(a_star(nodes[0], nodes[15]))

****Djikstra****
Processing (0, 0)
--adding: (0, 1) to heap
--adding: (1, 1) to heap
Processing (0, 1)
--adding: (1, 1) to heap
Processing (1, 1)
--adding: (2, 2) to heap
Processing (2, 2)
--adding: (1, 2) to heap
--adding: (2, 1) to heap
--adding: (3, 3) to heap
Processing (3, 3)
--adding: (3, 2) to heap
[(0, 0), (1, 1), (2, 2), (3, 3)]

****A Star****
Processing (0, 0)
--adding: (0, 1) to heap
--adding: (1, 1) to heap
Processing (1, 1)
--adding: (0, 1) to heap
--adding: (2, 2) to heap
Processing (2, 2)
--adding: (1, 2) to heap
--adding: (2, 1) to heap
--adding: (3, 3) to heap
Processing (3, 3)
--adding: (3, 2) to heap
[(0, 0), (1, 1), (2, 2), (3, 3)]
